In [1]:
import pandas as pd 
import os 

"""
path = pasta/caminho onde se encontra os arquivos do SRAG
saida = pasta onde será criado o arquivo agrupado 
municipios = arquivo com as informações de cada municipio segundo o IBGE
UF = arquivo com as informações de cada estado segundo o IBGE
"""

path = r"C:\Users\eluar\Documents\Estudo_IA_Python\AESOP_Projeto\Dados_AESOP\SRAG"
saida = r"C:\Users\eluar\Documents\Estudo_IA_Python\AESOP_Projeto\Dados_AESOP\SRAG_PROCESSADO"
municipios = pd.read_csv(r"C:\Users\eluar\Documents\Estudo_IA_Python\AESOP_Projeto\Dados_AESOP\DADOS_IBGE_MUNI\IBGE_MUNICIPIOS.csv", sep=",", encoding = 'utf-8-sig')
UF = pd.read_csv(r"C:\Users\eluar\Documents\Estudo_IA_Python\AESOP_Projeto\Dados_AESOP\DADOS_IBGE_ESTAGOS\IBGE_ESTADOS.csv", sep=",", encoding = 'utf-8-sig')

#Lista de arquivos na pasta path 
arquivos = os.listdir(path)

cont = 0

#Lista de váriaveis que serão agrupadas ao longo dos anos 
colunas = ['SG_UF_NOT','ID_MUNICIP','CO_MUN_NOT','SG_UF','ID_MN_RESI','CO_MUN_RES','DT_NOTIFIC','SEM_NOT','DT_SIN_PRI','SEM_PRI','CLASSI_FIN','CLASSI_OUT','CRITERIO','CS_SEXO','DT_NASC','CS_RACA']

In [12]:
#carrega os arquivos em partes para otimizar o uso de memória pelo pandas
def carregarArq(caminho, separ):
    chunks = []
    dados_chunks = pd.DataFrame()
    for chunk in pd.read_csv(caminho, sep= separ, encoding = 'iso-8859-1', chunksize=200000):
        chunks.append(chunk)
    dados_chunks = pd.concat(chunks)
    return dados_chunks


In [13]:
#Exclui as colunas do dataframe que não serão utilizadas
def excluirCols(data):
    
    for coluna in data.columns: 
        if coluna not in colunas:
            data.drop(coluna, axis = 1, inplace = True)
    return data  

In [14]:
#encontra o digito verificador do código do IBGE e retorna uma lista com todos os códigos dos municípios 
def RetCodigoIBGE(codigos):
    listcod = []
    for i in codigos: 
        codigo = str(i)
        #verifica se o codigo do municipio contem apenas numeros 
        if (codigo[:6].isdigit() == False):
            listcod.append('NaN')
        else:
            d1 = int(codigo[0])
            d2 = (int(codigo[1]) * 2) % 10 + (int(codigo[1]) * 2) // 10    
            d3 = int(codigo[2])
            d4 = (int(codigo[3]) * 2) % 10 + (int(codigo[3]) * 2) // 10
            d5 = int(codigo[4])
            d6 = (int(codigo[5]) * 2) % 10 + (int(codigo[5]) * 2) // 10
            digit = (10 - (d1 + d2 + d3 + d4 + d5 + d6) % 10) % 10
            cod = codigo[:6]+''+str(digit)
            listcod.append(cod)
    return listcod

In [15]:
# Retorna a semana de notificação sem o ano 
def retSemana(semanas):
    listSem = []
    for i in semanas:
        semana = str(i)
        if (semana.isdigit() == False or len(semana) < 4):
            listSem.append(semana)
        else:
            sem = semana[4:]
            sem = int(sem)
            listSem.append(sem)
    return listSem

In [16]:
# Retorna a sigla da UF a partir do código da UF 
def retUF(codigos, data):
    listUF = []
    UFIBGE = list(data['uf'])
    codUFIBGE = list(data['codigo_uf'])
    for cod in codigos:
        cod = str(cod)
        if(cod.isdigit() == True):
            if(int(cod) in codUFIBGE):
                pos = codUFIBGE.index(int(cod))
                listUF.append(UFIBGE[pos])
        else:
            listUF.append(cod)
    return listUF

In [17]:
# busca no dataframe e retorna nome, latitude e longitude dos municípios a partir da lista de códigos do IBGE
def retMunicipios(codigos, data):
    listmuni = []
    listlat = []
    listlon = []
    codIBGE = list(data['codigo_ibge'])
    muniIBGE = list(data['nome'])
    latIBGE = list(data['latitude'])
    lonIBGE = list(data['longitude'])
    
    for cod in codigos: 
        #verifica se o codigo do municipio contem apenas numeros para realizar a busca do nome do município e suas coordenadas 
        if (cod.isdigit() != False):
            if (int(cod) in codIBGE):
                pos = codIBGE.index(int(cod))
                listmuni.append(muniIBGE[pos])
                listlat.append(latIBGE[pos])
                listlon.append(lonIBGE[pos])
            #Quando não encontrar o município adicionar NAN (código errado)
            else:
                listmuni.append('NaN')
                listlat.append('NaN')
                listlon.append('NaN')
        #Quando o código não for um número adicionar NAN
        else:
            listmuni.append('NaN')
            listlat.append('NaN')
            listlon.append('NaN')
    return listmuni, listlat, listlon

In [18]:
df =  pd.DataFrame()
while (cont < len(arquivos)):
    
  
    caminho = path+'\\'+arquivos[cont]
    
    #Carrega os arquivos e exclui as colunas conforme os arquivos de 2009 a 2011 sep = ',',  após 2011 sep = ';'
    if (arquivos[cont] == 'INFLUD09.csv' or arquivos[cont] == 'INFLUD10.csv' or arquivos[cont] == 'INFLUD11.csv'):   
        sep = ','
        dados = carregarArq(caminho, sep)
        dados_col = excluirCols(dados)
    
    else:
        sep = ';'
        dados = carregarArq(caminho, sep)
        dados_col = excluirCols(dados)
       
    
    #verifica se os arquivos estão entre os anos de 2009 a 2018 para padronizar os estados e semanas com os registros de 2019 a 2022
    if (arquivos[cont] != 'INFLUD19.csv' or arquivos[cont] != 'INFLUD20.csv' or arquivos[cont] != 'INFLUD21.csv' or arquivos[cont] != 'INFLUD22.csv'):
        SM = retSemana(dados_col['SEM_NOT'])
        listaUF = retUF(dados_col['SG_UF_NOT'], UF)
        listaUF_PA = retUF(dados_col['SG_UF'], UF) 
        
        dados_col.drop(['SEM_NOT', 'SG_UF_NOT', 'SG_UF'], axis = 1, inplace = True)
        
        dados_col['SEM_NOT'] = SM
        dados_col['SG_UF_NOT'] = listaUF
        dados_col['SG_UF'] = listaUF_PA
              
    """ 
    Se o dataset não possuir os campos CO_MUN_NOT e CO_MUN_RES (2009 a 2018) valida os códigos do IBGE usando o ID_MUNICIP
    e o ID_MN_RESI depois padroniza seguindo as características da base mais atual de 2019 a 2022 
    """
    if ("CO_MUN_NOT" not in dados_col.columns or "CO_MUN_RES" not in dados_col.columns):
        COD_MUNI = RetCodigoIBGE(dados_col['ID_MUNICIP'])
        ID_MUNI, LAT_MUNI_AT,  LON_MUNI_AT = retMunicipios(COD_MUNI, municipios)
        
        COD_MN_RE = RetCodigoIBGE(dados_col['ID_MN_RESI'])
        ID_MUNI_RE, LAT_MUNI_RE_AT, LON_MUNI_RE_AT = retMunicipios(COD_MN_RE, municipios)
        
        dados_col.drop(['ID_MUNICIP', 'ID_MN_RESI'], axis = 1, inplace = True)
        
        dados_col['ID_MUNICIP'] = ID_MUNI
        dados_col['CO_MUN_NOT'] = COD_MUNI
        dados_col['LAT_MUNI'] = LAT_MUNI_AT
        dados_col['LON_MUNI'] = LON_MUNI_AT 
        
        dados_col['ID_MN_RESI'] = ID_MUNI_RE
        dados_col['CO_MUN_RES'] = COD_MN_RE
        dados_col['LAT_MUNI_RES'] = LAT_MUNI_RE_AT
        dados_col['LON_MUNI_RES'] = LON_MUNI_RE_AT
        
    else:
        
        COD_MUNI = RetCodigoIBGE(dados_col['CO_MUN_NOT'])
        ID_MUNI, LAT_MUNI_AT,  LON_MUNI_AT = retMunicipios(COD_MUNI, municipios)
        
        COD_MN_RE = RetCodigoIBGE(dados_col['CO_MUN_RES'])
        ID_MUNI_RE, LAT_MUNI_RE_AT, LON_MUNI_RE_AT = retMunicipios(COD_MN_RE, municipios)
        
        dados_col.drop(['ID_MUNICIP', 'ID_MN_RESI', 'CO_MUN_NOT', 'CO_MUN_RES'], axis = 1, inplace = True)
        
        dados_col['ID_MUNICIP'] = ID_MUNI
        dados_col['CO_MUN_NOT'] = COD_MUNI
        dados_col['LAT_MUNI'] = LAT_MUNI_AT
        dados_col['LON_MUNI'] = LON_MUNI_AT 
        
        dados_col['ID_MN_RESI'] = ID_MUNI_RE
        dados_col['CO_MUN_RES'] = COD_MN_RE
        dados_col['LAT_MUNI_RES'] = LAT_MUNI_RE_AT
        dados_col['LON_MUNI_RES'] = LON_MUNI_RE_AT
    
    # Convertemos o campo SEM_NOT para o formato 'int64' antes de agrupar os dados 
    if(dados_col['SEM_NOT'].dtype != 'int64'):
        dados_col['SEM_NOT'] = dados_col['SEM_NOT'].astype('int64')

    
    df = df.append(dados_col)
    
    cont = cont+1

arquivo_agrupado = saida+'\\SRAG_AGP.csv'
df.to_csv(arquivo_agrupado)
print("Arquivo Criado...")


c:\users\eluar\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (42,53,78,79,80,83) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\eluar\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (42,74,80,81) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\eluar\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (72,74) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\eluar\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (74,75) have mixed types.Specify dtype opt

Arquivo Criado...
